<a href="https://colab.research.google.com/github/matzz-11/Quantum_Colab.ipynb/blob/main/6_Po%C3%A7o_Quadrado_Duplo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Poço quadrado duplo


---

## Requisitos do Código

Além das conhecidas bibliotecas para manipulação e cálculos numéricos, geração de gráficos, interatividade e cálculo matricial;

- numpy
- matplotlib.pyplot
- ipywidgets
- scipy.sparse e scipy.linalg

Temos uma novidade dessa vez, que serve para limpar a saída anterior do notebook:

- IPython.display

Usamos essa nova biblioteca para facilitar a simulação conforme alteração dos parâmetros, pois agora temos mais opções de interação entre as funções de onda!

**NÃO SE ESQUEÇA DOS DESAFIOS PARA TREINAR APÓS O CÓDIGO!!**

---

## Potencial, Função de onda e Quantização da energia - Parâmetros Qualitativos

No início dos estudos, esse potencial é apresentado de forma qualitativa, para nos ajudar a desenvolver uma intuição física do que irá vir! Basicamente, temos 3 situações que devemos nos familiarizar nesse tópico.

- Distância de separação = 0
- Distância de separação → 0
- Distância de separação → ∞

Estamos interessados em ver como as funções de onda e densidades de probabilidade se comportam conforme essa distância se altera. No primeiro caso, é como se tivessemos um grande poço quadrado finito, "com o dobro de largura", sendo análogo ao estudo anterior, apenas com mais espaço!

No segundo caso, com os poços separados, temos uma "abertura" na região, resultado no **tunelamento** entre poços! Esse é um dos casos mais interessantes, pois a probabilidade de tunelar irá depender da energia da partícula estar condizendo com os níveis quantizados de cada poço!

No terceiro caso, aprendemos o fenômeno da **degenerescência**, onde os poços estão tão distantes que começam a se comportar como "únicos", sem acoplamentos. Isso faz com que ambos "convirgam" para a **mesma energia**, com as funções de onda organizadas de forma simétrica.

---

## Aplicações

Uma das aplicações para o poço quadrado duplo é a **ligação molecular simples**. Temos o H₂ como exemplo, onde cada poço representaria um núcleo de hidrogênio. Dessa forma, vemos que o elétron (representado matematicamente pela função de onda) teria maior probabilidade de estar em um núcleo ou outro, ou até mesmo "tunelando" entre ambos!

Além disso, é também utilizado para fabricação de **filtros de energia eletrônica!** Montamos um sistema "sanduíche" de barreiras e poços, de forma que **apenas elétrons com energia condizente com os níveis quantizados do poço irão ter maior probabilidade de atravessar a barreira**, dessa maneira, criamos um sistema que pode ser usado em filtros de injeção de energia em transistores, dispositivos de detecção de energia específica e fontes de corrente seletiva.

---

## Conclusão

Sendo assim, apresentamos dois gráficos, um estático mostrando vários níveis de energia e outro dinâmico, onde é possível alterar parâmetros e ver o que acontece! Segue a legenda de cores e parâmetros:

Roxo - Função de onda

Amarelo - Densidade de Probabilidade

Azul - Nível atual de energia

Verde tracejado - Níveis anteriores e posteriores de energia

Preto tracejado - Barreiras/Contorno dos poços

"a" - Largura dos poços

"V₀" - Energia potencial

"b" - Distância entre os poços

"n" - Número quântico

---

##Código

In [ ]:
!pip install ipywidgets

#Bibliotecas
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.linalg import eigh
import ipywidgets as widgets
from IPython.display import display, clear_output

#Definição da função do Poço Duplo
def double_well_H(a, V0, b, xmin=-10, xmax=10, dx=0.05, m=1.0, hbar=1.0):
    x = np.arange(xmin, xmax+dx, dx)
    V = np.zeros_like(x)
    well1 = (x >= -b/2 - a/2) & (x <= -b/2 + a/2)
    well2 = (x >=  b/2 - a/2) & (x <=  b/2 + a/2)
    V[well1 | well2] = -V0
    N = x.size
    diag0 = -2 * np.ones(N)
    diag1 = np.ones(N-1)
    D2 = diags([diag1, diag0, diag1], offsets=[-1,0,1]) / dx**2
    T = -(hbar**2)/(2*m) * D2
    H = T.toarray() + np.diag(V)
    E, psi = eigh(H)
    bound = E < 0
    return x, V, E[bound], psi[:, bound]

def plot_double_well(a, V0, n, b, mostrar):
    clear_output(wait=True)

#Definição dos estados ligados e potencial
    x, V, E, psi = double_well_H(a, V0, b)
    N = len(E)
    if N == 0:
        print("⚠️ Sem estados ligados. Aumente a ou V₀."); return
    if n > N:
        print(f"⚠️ Só há {N} níveis. Selecione n ≤ {N}."); return

    dx = x[1] - x[0]
    psi = psi / np.sqrt((psi**2).sum(axis=0)*dx)[None,:]
    idx = n-1
    E_n  = E[idx]
    amp  = V0 * 0.3
    psi_n    = psi[:, idx]
    psi_plot = psi_n * amp + E_n
    rho_plot = psi_n**2 * amp + E_n

    fig, (ax1, ax2) = plt.subplots(1,2,figsize=(16,6))

    ax1.plot(x, V, 'k--', label='Potencial')

#Energia do nível anterior
    if idx-1 >= 0:
        E_lo = E[idx-1]
        ax1.hlines(E_lo, x[0], x[-1],
                   colors='green', linestyles='--',
                   label=f'Eₙ₋₁ = {E_lo:.3f}')

#Energia do nível atual
    ax1.hlines(E_n, x[0], x[-1],
               colors='blue', linewidth=2,
               label=f'Eₙ    = {E_n:.3f}')

#Energia do nível posterior
    if idx+1 < N:
        E_hi = E[idx+1]
        ax1.hlines(E_hi, x[0], x[-1],
                   colors='green', linestyles='-.',
                   label=f'Eₙ₊₁ = {E_hi:.3f}')

#Botões para alternar entre função de onda, densidade de probabilidade ou ambos
    if mostrar in ['Função de onda','Ambos']:
        ax1.plot(x, psi_plot, color='purple', label='Ψₙ(x)')
    if mostrar in ['Densidade','Ambos']:
        ax1.plot(x, rho_plot, color='orange', label='|Ψₙ|²')

    ax1.set_title('Poço Duplo (Interativo)')
    ax1.set_xlabel('x'); ax1.set_ylabel('Energia')
    ax1.legend(loc='upper right')

#Gráfico estático
    a_s, b_s = 1.5, 3.0
    x_s, V_s, E_s, psi_s = double_well_H(a_s, V0, b_s)
    dx_s = x_s[1] - x_s[0]
    psi_s = psi_s / np.sqrt((psi_s**2).sum(axis=0)*dx_s)[None,:]
    amp_s = V0 * 0.3

    ax2.plot(x_s, V_s, 'k--', label='Potencial')
    for i, Ei in enumerate(E_s[:3]):
        ax2.hlines(Ei, x_s[0], x_s[-1],
                   colors='blue', label=f'E{i+1} = {Ei:.3f}')
        psi_i = psi_s[:, i]
        ax2.plot(x_s, psi_i * amp_s + Ei,
                 color='purple', alpha=0.7)
        ax2.plot(x_s, psi_i**2 * amp_s + Ei,
                 color='orange', alpha=0.5)

    ax2.set_title(f'Estático: a={a_s}, b={b_s}')
    ax2.set_xlabel('x'); ax2.set_ylabel('Energia')
    ax2.legend(bbox_to_anchor=(1.05,1), loc='upper left')

    plt.tight_layout()
    plt.show()

#Interatividade
a_slider = widgets.FloatSlider(min=0.5, max=4.0, step=0.1,
                               value=1.0, description='Largura a')
V0_slider = widgets.FloatSlider(min=1.0, max=30.0, step=1.0,
                                value=20.0, description='V₀')
b_slider = widgets.FloatSlider(min=0.1, max=4.0, step=0.1,
                               value=1.0, description='Distância b')
n_slider = widgets.IntSlider(min=1, max=5, step=1,
                             value=1, description='Nível n')
mode_tog = widgets.ToggleButtons(
    options=['Função de onda','Densidade','Ambos'],
    description='Mostrar:')

ui  = widgets.VBox([a_slider, V0_slider, b_slider, n_slider, mode_tog])
out = widgets.interactive_output(
    plot_double_well,
    {'a':a_slider,'V0':V0_slider,
     'n':n_slider,'b':b_slider,
     'mostrar':mode_tog}
)

display(ui, out)


## Desafios para fixação

---

### Exercício 1 — Variação de Energia entre níveis vs Espaçamento dos poços

Com um valor de "n" fixo, varie "b" de 0.2 até 4.0 com passos arbitrários. Anote os valores de "E_n" e "E_n+1" e calcule a diferença entre eles ΔE = E_(n+1) - E_(n). Esboce um gráfico de ΔE x b.

> 1. O que ocorre com ΔE conforme b → 4?
> 2. Existe um valor de b em que ΔE praticamente zera? Como você relaciona isso ao regime de dois poços isolados?

---

### Exercício 2 — Poço Duplo → Poço Quadrado Finito

Deixe os valores de "a" fixo em 1.0, "V₀" fixo em 20 e "n" fixo em 2.0. Varie o valor de "b" de 4 até 1.5, de forma decrescente.

> 1. Conforme b → 0, o que podemos ressaltar sobre o comportamento da função de onda?
> 2. O que ocorre com ΔE conforme b → 0?

---
